## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.60,61,11,11.05,few clouds
1,1,Georgetown,MY,5.4112,100.3354,82.33,93,20,3.44,few clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,78.55,96,26,1.01,scattered clouds
3,3,Barrow,US,71.2906,-156.7887,-7.92,93,6,17.27,clear sky
4,4,Kanchanaburi,TH,14.6333,99.1000,75.70,71,99,7.61,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.60,61,11,11.05,few clouds
1,1,Georgetown,MY,5.4112,100.3354,82.33,93,20,3.44,few clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,78.55,96,26,1.01,scattered clouds
4,4,Kanchanaburi,TH,14.6333,99.1000,75.70,71,99,7.61,overcast clouds
5,5,Vaini,TO,-21.2000,-175.2000,84.36,79,75,4.61,broken clouds
12,12,Conde,BR,-7.2597,-34.9075,78.26,89,40,3.44,scattered clouds
13,13,Hithadhoo,MV,-0.6000,73.0833,82.80,71,3,8.43,clear sky
14,14,Kapaa,US,22.0752,-159.3190,80.58,95,32,4.00,moderate rain
15,15,Faanui,PF,-16.4833,-151.7500,82.20,68,100,1.16,overcast clouds
17,17,Kahului,US,20.8947,-156.4700,76.84,80,59,14.00,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.60,few clouds,-23.1203,-134.9692,
1,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
2,Puerto Ayora,EC,78.55,scattered clouds,-0.7393,-90.3518,
4,Kanchanaburi,TH,75.70,overcast clouds,14.6333,99.1000,
5,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
12,Conde,BR,78.26,scattered clouds,-7.2597,-34.9075,
13,Hithadhoo,MV,82.80,clear sky,-0.6000,73.0833,
14,Kapaa,US,80.58,moderate rain,22.0752,-159.3190,
15,Faanui,PF,82.20,overcast clouds,-16.4833,-151.7500,
17,Kahului,US,76.84,broken clouds,20.8947,-156.4700,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
hotel_df.dtypes

City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!= '')]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.60,few clouds,-23.1203,-134.9692,People ThankYou
1,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
2,Puerto Ayora,EC,78.55,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
4,Kanchanaburi,TH,75.70,overcast clouds,14.6333,99.1000,แพบุญศรี
5,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
652,Pakaur,IN,79.03,few clouds,24.6333,87.8500,Jannisar Ansari
658,Cabangahan,PH,83.39,scattered clouds,9.0781,122.9469,Centralle District Beach Resort
665,Aquiraz,BR,80.69,overcast clouds,-3.9014,-38.3911,Pousada Rua Camurupim
682,Manta,EC,78.58,scattered clouds,-0.9500,-80.7333,Balandra Hotel


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))